README.md

In [11]:
readme_content = """
# Algo Trading Bot

This is a fully-featured algorithmic trading system for Indian stocks using yfinance, technical indicators, ML-based predictions, Google Sheets logging, and Telegram notifications. It is designed to be run in Google Colab.

## Features

- **Fetches historical data** from Yahoo Finance for a configurable list of stocks.
- **Calculates technical indicators** (RSI, MACD, SMA, Bollinger Bands, ATR, VWAP, Stochastic).
- **Backtests a trading strategy** (relaxed RSI-based demo for testing).
- **Logs all trades and summary stats** to Google Sheets.
- **Sends trade/summary notifications** to Telegram.
- **Trains a RandomForest ML model** to predict next-day price movement and logs results.
- **Interactive Plotly charts** for each stock.
- **Robust error handling** and logging.

## Setup

1. **Clone this repo or download as ZIP.**
2. **Install requirements** (Colab cell does this, or use pip locally):
3. **Google Sheets API:**
- Create a Google Service Account and download the JSON credentials.
- Upload your JSON credentials to Colab or your project directory.
- Share your Google Sheet with the service account email.

4. **Edit `config` in `algo_trading_bot.py`:**
- Set `GDRIVE_JSON` to the path of your service account JSON.
- Set your Telegram Bot token and Chat ID.

5. **Run in Colab or locally:**
- For Colab: Copy/paste the code, upload your JSON, and run.
- For local: Ensure all dependencies are installed.

## Usage

- Adjust the list of stocks and strategy as desired.
- All trades, summary, and ML analytics will be logged to your configured Google Sheet.
- Telegram notifications will be sent automatically.

## Files

- `algo_trading_bot.py` — Main script (copy-paste into Colab or run locally)
- `README.md` — This file
- *(Optional)* `requirements.txt` — List of Python dependencies

## License

MIT
"""

with open("README.md", "w") as f:
 f.write(readme_content)

print("✅ README.md file saved.")


✅ README.md file saved.


AlgoTradingBot.ipynb

In [10]:
# --- 1. Install Required Libraries ---
# !pip install -U -q yfinance ta gspread gspread_dataframe oauth2client scikit-learn python-telegram-bot==13.7 plotly

# --- 2. Imports and Setup ---
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import MACD, SMAIndicator
from ta.volatility import BollingerBands, AverageTrueRange
from ta.volume import VolumeWeightedAveragePrice
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import gspread_dataframe as gd
import logging
import requests
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from google.colab import files, drive
from plotly.offline import init_notebook_mode
import plotly.io as pio
import os

init_notebook_mode(connected=True)
pio.renderers.default = "colab"

# --- 3. Configuration ---
class Config:
    STOCKS = [
        "RELIANCE.NS", "TCS.NS", "HDFCBANK.NS", "INFY.NS", "ICICIBANK.NS",
        "SBIN.NS", "KOTAKBANK.NS"
    ]
    INTERVAL = "1d"
    LOOKBACK_DAYS = 200 # For backtesting for approximately 6 months (180 days + buffer)
    INITIAL_CAPITAL = 100000
    RISK_PER_TRADE = 0.01 # 1% of current capital risked per trade
    COMMISSION = 0.0005 # 0.05% commission on trade value (both buy and sell)
    SHEET_NAME = "AlgoTradingLog"
    TRADE_LOG_TAB = "TradeLog"
    SUMMARY_TAB = "Summary"
    ANALYTICS_TAB = "Analytics"
    # IMPORTANT: Ensure this JSON file is in the root of your mounted Google Drive
    GDRIVE_JSON = "/content/uplifted-oxide-455216-e0-3691ecf11370 (1).json"
    ML_FEATURES = [ # Features for ML model
        "RSI", "MACD", "Signal", "Volume", "20DMA", "50DMA", "VWAP",
        "BB_High", "BB_Low", "ATR", "Stoch_K", "Stoch_D"
    ]
    TELEGRAM_TOKEN = "7955736257:AAGhidPFAPRfLWY9d3A7Ck9Yw9Ed9r60j5g" # Replace with your bot token
    TELEGRAM_CHAT_ID = "5782710416" # Replace with your chat ID
    LOG_FILE = "/content/algo_trading.log"
    # Define the expected column order for the TradeLog sheet
    TRADE_LOG_COLUMNS_ORDER = [
        "Date", "Symbol", "Type", "Price", "Shares", "PnL", "Capital"
    ]

config = Config()

# Mount Google Drive and set up logging
# This section assumes you've run drive.mount() and the JSON file is accessible.
drive.mount('/content/drive', force_remount=True)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(config.LOG_FILE),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# --- 4. Helper Functions ---

def fetch_stock_data(symbol, interval, lookback_days):
    """
    Fetches historical stock data for a given symbol.
    Automatically adjusts for splits and dividends by setting auto_adjust=True.
    Handles MultiIndex columns and standardizes column names.
    """
    try:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=lookback_days)
        logger.info(f"Fetching data for {symbol} from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

        df = yf.download(
            symbol,
            start=start_date,
            end=end_date,
            interval=interval,
            progress=False,
            auto_adjust=True # Important for adjusted prices
        )

        if not isinstance(df, pd.DataFrame):
            logger.error(f"yf.download did not return a DataFrame for {symbol}. Returned type: {type(df)}. Cannot process data.")
            return None

        if df.empty:
            logger.warning(f"No data returned for {symbol} for the specified period.")
            return None

        # --- START OF FIX: Handle MultiIndex columns from yfinance ---
        if isinstance(df.columns, pd.MultiIndex):
            # For a single symbol download, yfinance often returns columns like ('Close', 'SYMBOL')
            # We want to keep only the metric part (e.g., 'Close', 'Open', 'Volume').
            df.columns = df.columns.get_level_values(0)
            logger.info(f"Flattened MultiIndex columns for {symbol}. New column names: {df.columns.tolist()}")

        # Standardize column names (e.g., 'Adj Close' to 'Close', and remove spaces)
        new_columns = []
        for col in df.columns:
            cleaned_col = str(col).replace(' ', '_') # Replace spaces with underscores
            new_columns.append(cleaned_col)
        df.columns = new_columns

        # Ensure 'Close' column refers to the adjusted close if 'Adj_Close' is present
        if 'Adj_Close' in df.columns:
            if 'Close' in df.columns and not df['Close'].equals(df['Adj_Close']):
                # If both exist and are different, prefer 'Adj_Close' as the final 'Close'
                df['Close'] = df['Adj_Close']
                logger.info(f"Overwriting 'Close' with 'Adj_Close' for {symbol}.")
            elif 'Close' not in df.columns:
                # If only 'Adj_Close' exists, rename it to 'Close'
                df = df.rename(columns={'Adj_Close': 'Close'})
                logger.info(f"Renamed 'Adj_Close' to 'Close' for {symbol}.")

        # Filter the DataFrame to only include the required OHLCV columns
        required_ohlcv = ['Open', 'High', 'Low', 'Close', 'Volume']
        df_filtered = pd.DataFrame(index=df.index) # Create a new DF to populate with correct columns

        for col in required_ohlcv:
            if col in df.columns:
                df_filtered[col] = df[col]
            else:
                logger.warning(f"Required column '{col}' not found in data for {symbol}.")

        df = df_filtered.dropna() # Drop rows that might have NaN due to missing columns or other issues
        df.index = pd.to_datetime(df.index) # Ensure index is datetime

        # --- END OF FIX ---

        logger.info(f"Successfully fetched and processed data for {symbol}. Shape: {df.shape}. Columns: {df.columns.tolist()}")
        return df
    except Exception as e:
        logger.error(f"Error fetching data for {symbol}: {str(e)}")
        return None

def calculate_technical_indicators(df, symbol):
    """
    Calculates various technical indicators for the given DataFrame.
    Includes RSI, MACD, Moving Averages, Bollinger Bands, ATR, VWAP, and Stochastic Oscillator.
    """
    if df is None or df.empty:
        logger.warning(f"Empty or None DataFrame passed to calculate_technical_indicators for {symbol}. Cannot calculate indicators.")
        return None

    try:
        df_copy = df.copy() # Operate on a copy to avoid SettingWithCopyWarning

        # Verify required columns for indicator calculation
        required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
        if not all(col in df_copy.columns for col in required_cols):
            missing = [col for col in required_cols if col not in df_copy.columns]
            logger.error(f"Missing required columns for indicator calculation for {symbol}: {missing}. Found: {df_copy.columns.tolist()}")
            return None

        # Assign series for clarity
        close = df_copy['Close']
        high = df_copy['High']
        low = df_copy['Low']
        volume = df_copy['Volume']

        # Trend indicators
        df_copy["20DMA"] = SMAIndicator(close=close, window=20).sma_indicator() #
        df_copy["50DMA"] = SMAIndicator(close=close, window=50).sma_indicator() #

        # Momentum indicators
        df_copy["RSI"] = RSIIndicator(close=close, window=14).rsi() #
        macd = MACD(close=close)
        df_copy["MACD"] = macd.macd() #
        df_copy["Signal"] = macd.macd_signal() #

        # Volume indicators
        df_copy["VWAP"] = VolumeWeightedAveragePrice(
            high=high,
            low=low,
            close=close,
            volume=volume
        ).volume_weighted_average_price() #

        # Volatility indicators
        bb = BollingerBands(close=close, window=20, window_dev=2)
        df_copy["BB_High"] = bb.bollinger_hband() #
        df_copy["BB_Low"] = bb.bollinger_lband() #
        df_copy["ATR"] = AverageTrueRange(high=high, low=low, close=close, window=14).average_true_range() #

        # Stochastic oscillator
        stoch = StochasticOscillator(high=high, low=low, close=close, window=14, smooth_window=3)
        df_copy["Stoch_K"] = stoch.stoch() #
        df_copy["Stoch_D"] = stoch.stoch_signal() #

        logger.info(f"Successfully calculated technical indicators for {symbol}.")
        return df_copy.dropna() # Drop rows where indicators could not be calculated (due to rolling window)
    except Exception as e:
        logger.error(f"Error calculating indicators for {symbol}: {str(e)}")
        return None

def prepare_dataset(symbols, interval, lookback_days):
    """
    Prepares the dataset by fetching data and calculating indicators for all symbols.
    """
    data = {}
    for symbol in symbols: #
        logger.info(f"--- Processing {symbol} ---")
        df = fetch_stock_data(symbol, interval, lookback_days) #
        if df is not None:
            df = calculate_technical_indicators(df, symbol)
            if df is not None:
                data[symbol] = df
        time.sleep(1)   # Be gentle with the API to avoid being blocked
    return data

class TradingStrategy:
    """
    Defines the trading strategy, including signal generation and trade execution.
    """
    def __init__(self, initial_capital=config.INITIAL_CAPITAL,
                 risk_per_trade=config.RISK_PER_TRADE,
                 commission=config.COMMISSION):
        self.initial_capital = initial_capital
        self.current_capital = initial_capital
        self.risk_per_trade = risk_per_trade
        self.commission = commission
        logger.info(f"TradingStrategy initialized with capital: {initial_capital}, risk/trade: {risk_per_trade}, commission: {commission}")

    def generate_signals(self, df):
        """
        Generates buy and sell signals based on simplified RSI conditions for demonstration.
        For actual trading, combine with other indicators as per your strategy.
        Buy: RSI crosses below 40 (oversold)
        Sell: RSI crosses above 60 (overbought)
        """
        if df is None or df.empty:
            logger.warning("Empty or None DataFrame passed to generate_signals. No signals generated.")
            return None
        try:
            df_copy = df.copy() # Operate on a copy to avoid SettingWithCopyWarning

            # Check if 'RSI' is present
            if 'RSI' not in df_copy.columns:
                logger.error("RSI column not found for signal generation. Cannot generate signals.")
                return None

            # Initialize signals to False
            df_copy['Signal_Buy'] = False
            df_copy['Signal_Sell'] = False

            # Ensure we have at least two data points for cross-over logic
            if len(df_copy) > 1:
                # Buy signal: RSI crosses below 40
                # Check for (previous_RSI >= 40) AND (current_RSI < 40)
                df_copy['Signal_Buy'] = (df_copy['RSI'].shift(1) >= 40) & (df_copy['RSI'] < 40)

                # Sell signal: RSI crosses above 60
                # Check for (previous_RSI <= 60) AND (current_RSI > 60)
                df_copy['Signal_Sell'] = (df_copy['RSI'].shift(1) <= 60) & (df_copy['RSI'] > 60)

            logger.info(f"Signals generated. Buy signals: {df_copy['Signal_Buy'].sum()}, Sell signals: {df_copy['Signal_Sell'].sum()}")

            return df_copy
        except Exception as e:
            logger.error(f"Error generating signals: {str(e)}")
            return None

    def execute_trades(self, symbol, df):
        """
        Executes simulated trades based on generated signals and manages capital.
        Logs trades for P&L tracking.
        """
        if df is None or df.empty:
            logger.warning(f"Empty or None DataFrame passed to execute_trades for {symbol}. No trades executed.")
            return pd.DataFrame()

        trades = []
        position = 0    # 0 = no position, 1 = long position
        entry_price = 0
        shares = 0

        # Ensure index is datetime for iteration
        if not isinstance(df.index, pd.DatetimeIndex):
            df.index = pd.to_datetime(df.index)

        for idx, row in df.iterrows():
            try:
                current_price = row['Close']

                # Exit logic
                if row['Signal_Sell'] and position == 1:
                    exit_price = current_price
                    pnl = (exit_price - entry_price) * shares
                    # Deduct commission for both buy and sell
                    pnl -= (entry_price + exit_price) * shares * self.commission

                    trades.append({
                        "Symbol": symbol,
                        "Date": idx, # Pass the datetime object directly
                        "Type": "SELL",
                        "Price": round(exit_price, 2),
                        "Shares": shares,
                        "PnL": round(pnl, 2),
                        "Capital": round(self.current_capital + pnl, 2)
                    })

                    self.current_capital += pnl
                    position = 0
                    shares = 0
                    logger.info(f"Exited position in {symbol} at {exit_price:.2f} (PnL: {pnl:.2f}) on {idx.strftime('%Y-%m-%d')}. Current Capital: {self.current_capital:.2f}")

                # Entry logic
                if row['Signal_Buy'] and position == 0 and self.current_capital > 0:
                    entry_price = current_price

                    # Define a fixed stop loss percentage for risk calculation
                    # This determines the maximum loss per share we are willing to take for share calculation.
                    stop_loss_pct = 0.03 # Example: 3% stop loss from entry price for risk calculation

                    potential_loss_per_share = entry_price * stop_loss_pct

                    if potential_loss_per_share <= 0:
                        logger.warning(f"Calculated potential loss per share for {symbol} at {idx} is zero or negative ({potential_loss_per_share:.2f}). Adjusting shares to a safe amount.")
                        # Fallback: buy a small fixed number of shares if risk calculation is problematic
                        # Ensure we can afford at least 1 share including commission
                        shares = int(self.current_capital / (entry_price * (1 + self.commission))) if entry_price > 0 else 0
                        if shares == 0 and self.current_capital >= entry_price * (1 + self.commission):
                            shares = 1
                    else:
                        # Calculate shares based on risk per trade (e.g., 1% of capital)
                        risk_amount = self.current_capital * self.risk_per_trade
                        shares = int(risk_amount / potential_loss_per_share)

                    # Ensure we don't exceed available capital (including commission)
                    trade_cost = entry_price * shares * (1 + self.commission)
                    if trade_cost > self.current_capital:
                        shares = int(self.current_capital / (entry_price * (1 + self.commission))) # Recalculate based on max affordable shares
                        trade_cost = entry_price * shares * (1 + self.commission) # Update trade_cost

                    if shares > 0:
                        trades.append({
                            "Symbol": symbol,
                            "Date": idx, # Pass the datetime object directly
                            "Type": "BUY",
                            "Price": round(entry_price, 2),
                            "Shares": shares,
                            "PnL": 0, # PnL is 0 at entry
                            "Capital": round(self.current_capital - trade_cost, 2)
                        })

                        self.current_capital -= trade_cost
                        position = 1
                        logger.info(f"Entered position in {symbol} at {entry_price:.2f} with {shares} shares (Cost: {trade_cost:.2f}) on {idx.strftime('%Y-%m-%d')}. Current Capital: {self.current_capital:.2f}")
                    else:
                        logger.info(f"Not enough capital or invalid share calculation to enter position for {symbol} at {idx}. Current Capital: {self.current_capital:.2f}, Price: {entry_price:.2f}")

            except Exception as e:
                logger.error(f"Trade execution error at {idx} for {symbol}: {str(e)}")
                continue

        # Close any remaining open position at the very end of the backtest period
        if position == 1 and not df.empty:
            exit_price = df.iloc[-1]['Close']
            pnl = (exit_price - entry_price) * shares
            pnl -= (entry_price + exit_price) * shares * self.commission

            trades.append({
                "Symbol": symbol,
                "Date": df.index[-1], # Pass the datetime object directly
                "Type": "SELL",
                "Price": round(exit_price, 2),
                "Shares": shares,
                "PnL": round(pnl, 2),
                "Capital": round(self.current_capital + pnl, 2)
            })

            self.current_capital += pnl
            logger.info(f"Closed final position in {symbol} at {exit_price:.2f} (PnL: {pnl:.2f}) on {df.index[-1].strftime('%Y-%m-%d')}. Final Capital: {self.current_capital:.2f}")

        return pd.DataFrame(trades)

def calculate_performance_metrics(trades_df):
    """
    Calculates various performance metrics from a DataFrame of trades.
    """
    if trades_df.empty:
        logger.info("No trades to calculate performance metrics for. Returning default zeros.")
        return {
            "Total Trades": 0, "Winning Trades": 0, "Losing Trades": 0, "Win Rate": 0,
            "Total PnL": 0, "PnL %": 0, "Max Drawdown": 0, "Profit Factor": 0,
            "Avg Win": 0, "Avg Loss": 0
        }

    try:
        # Filter for sell trades (which represent closed positions and realized PnL)
        sell_trades = trades_df[trades_df["Type"] == "SELL"].copy()

        # Ensure PnL is numeric
        sell_trades['PnL'] = pd.to_numeric(sell_trades['PnL'], errors='coerce').fillna(0)

        total_trades = len(sell_trades)
        winning_trades = len(sell_trades[sell_trades["PnL"] > 0])
        losing_trades = total_trades - winning_trades
        win_rate = winning_trades / total_trades if total_trades > 0 else 0
        total_pnl = sell_trades["PnL"].sum()
        pnl_pct = (total_pnl / config.INITIAL_CAPITAL) * 100 if config.INITIAL_CAPITAL > 0 else 0

        # Calculate max drawdown
        if not trades_df.empty and 'Capital' in trades_df.columns:
            # Ensure Capital is numeric and fill initial capital for drawdown start
            trades_df['Capital'] = pd.to_numeric(trades_df['Capital'], errors='coerce').fillna(config.INITIAL_CAPITAL)

            # Prepend initial capital to the capital series for accurate drawdown calculation
            capital_series = pd.Series([config.INITIAL_CAPITAL] + trades_df["Capital"].tolist())
            peak = capital_series.expanding(min_periods=1).max()
            drawdown = (peak - capital_series) / peak
            max_drawdown = drawdown.max() * 100 if not drawdown.empty else 0
        else:
            max_drawdown = 0

        # Calculate profit factor
        gross_profit = sell_trades[sell_trades["PnL"] > 0]["PnL"].sum()
        gross_loss = abs(sell_trades[sell_trades["PnL"] < 0]["PnL"].sum())
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf') # Handle division by zero

        # Calculate average win/loss
        avg_win = sell_trades[sell_trades["PnL"] > 0]["PnL"].mean() if winning_trades > 0 else 0
        avg_loss = sell_trades[sell_trades["PnL"] < 0]["PnL"].mean() if losing_trades > 0 else 0

        metrics = {
            "Total Trades": total_trades,
            "Winning Trades": winning_trades,
            "Losing Trades": losing_trades,
            "Win Rate": round(win_rate, 4), #
            "Total PnL": round(total_pnl, 2), #
            "PnL %": round(pnl_pct, 2),
            "Max Drawdown": round(max_drawdown, 2),
            "Profit Factor": round(profit_factor, 2),
            "Avg Win": round(avg_win, 2),
            "Avg Loss": round(avg_loss, 2)
        }
        logger.info(f"Performance metrics calculated: {metrics}")
        return metrics
    except Exception as e:
        logger.error(f"Error calculating performance metrics: {str(e)}")
        return {
            "Total Trades": 0, "Winning Trades": 0, "Losing Trades": 0, "Win Rate": 0,
            "Total PnL": 0, "PnL %": 0, "Max Drawdown": 0, "Profit Factor": 0,
            "Avg Win": 0, "Avg Loss": 0
        }

def train_ml_model(df, symbol):
    """
    Trains a RandomForestClassifier model to predict next day's price movement.
    Outputs prediction accuracy.
    """
    if df is None or df.empty:
        logger.warning(f"Empty or None DataFrame passed to train_ml_model for {symbol}. Skipping ML training.")
        return None, 0.0, None

    try:
        df_copy = df.copy() # Operate on a copy
        # Create target variable: 1 if next day's close is higher, 0 otherwise
        df_copy["Target"] = (df_copy["Close"].shift(-1) > df_copy["Close"]).astype(int)
        df_copy = df_copy.dropna() # Drop the last row where Target is NaN

        # Check for required features
        actual_features = [f for f in config.ML_FEATURES if f in df_copy.columns]
        missing_features = [f for f in config.ML_FEATURES if f not in df_copy.columns]
        if missing_features:
            logger.warning(f"Missing features for ML for {symbol}: {missing_features}. Using available features: {actual_features}")

        if not actual_features:
            logger.warning(f"No valid features available for ML model training for {symbol}. Skipping ML training.")
            return None, 0.0, None

        X = df_copy[actual_features]
        y = df_copy["Target"]

        # Check if we have enough data and both classes for training
        if len(df_copy) < 30: # Arbitrary minimum data points for training
            logger.warning(f"Insufficient data ({len(df_copy)} rows) for ML for {symbol}. Need at least 30 rows. Skipping ML model training.")
            return None, 0.0, None
        if len(y.unique()) < 2:
            logger.warning(f"Only one class ({y.unique()}) present in target variable for {symbol}. Cannot train classification model. Skipping ML model training.")
            return None, 0.0, None

        # Split data into training and testing sets
        split_idx = int(len(X) * 0.8)
        X_train, X_test = X[:split_idx], X[split_idx:]
        y_train, y_test = y[:split_idx], y[split_idx:]

        if X_test.empty or X_train.empty:
            logger.warning(f"Empty train or test set after split for {symbol}. Skipping ML model training.")
            return None, 0.0, None

        # Train RandomForestClassifier model
        model = RandomForestClassifier(
            n_estimators=200,
            max_depth=6,
            min_samples_split=10,
            random_state=42,
            n_jobs=-1 # Use all available cores
        )
        model.fit(X_train, y_train)

        # Evaluate model performance
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred) #
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)

        logger.info(f"ML model trained for {symbol} with accuracy: {accuracy:.2f}")
        logger.debug(f"Classification Report for {symbol}:\n{classification_report(y_test, y_pred, zero_division=0)}")
        return model, accuracy, report

    except Exception as e:
        logger.error(f"Error in ML training for {symbol}: {str(e)}")
        return None, 0.0, None

def plot_trading_signals(symbol, df, trades_df):
    """
    Generates an interactive Plotly graph showing price, indicators, and trade signals.
    """
    if df is None or df.empty:
        logger.warning(f"Empty or None DataFrame passed to plot_trading_signals for {symbol}. Cannot generate plot.")
        return None

    try:
        # Create subplots: one for price/MAs/BB, one for RSI/Stochastic
        fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                            vertical_spacing=0.05,
                            row_heights=[0.6, 0.2, 0.2])

        # Plot Close Price
        fig.add_trace(go.Scatter(x=df.index, y=df['Close'], name="Close Price", mode='lines', line=dict(color='blue')), row=1, col=1)

        # Plot Moving Averages if they exist
        if '20DMA' in df.columns:
            fig.add_trace(go.Scatter(x=df.index, y=df['20DMA'], name="20DMA", line=dict(color='orange', width=1)), row=1, col=1)
        if '50DMA' in df.columns:
            fig.add_trace(go.Scatter(x=df.index, y=df['50DMA'], name="50DMA", line=dict(color='purple', width=1)), row=1, col=1)

        # Plot Bollinger Bands if they exist
        if 'BB_High' in df.columns and 'BB_Low' in df.columns:
            fig.add_trace(go.Scatter(x=df.index, y=df['BB_High'], name="BB High", line=dict(color='gray', dash='dot', width=0.8)), row=1, col=1)
            fig.add_trace(go.Scatter(x=df.index, y=df['BB_Low'], name="BB Low", line=dict(color='gray', dash='dot', width=0.8), fill='tonexty', fillcolor='rgba(128,128,128,0.1)', showlegend=False), row=1, col=1)

        # Plot Trade markers
        if not trades_df.empty:
            # Ensure 'Date' column in trades_df is datetime for proper plotting alignment
            trades_df['Date'] = pd.to_datetime(trades_df['Date'])

            buy_trades = trades_df[trades_df["Type"] == "BUY"]
            sell_trades = trades_df[trades_df["Type"] == "SELL"]

            if not buy_trades.empty:
                fig.add_trace(go.Scatter(
                    x=buy_trades["Date"], y=buy_trades["Price"],
                    mode="markers", marker=dict(color='green', size=10, symbol='triangle-up', line=dict(width=1, color='DarkSlateGrey')),
                    name="Buy Signal"), row=1, col=1 # Moved row/col outside go.Scatter
                )

            if not sell_trades.empty:
                fig.add_trace(go.Scatter(
                    x=sell_trades["Date"], y=sell_trades["Price"],
                    mode="markers", marker=dict(color='red', size=10, symbol='triangle-down', line=dict(width=1, color='DarkSlateGrey')),
                    name="Sell Signal"), row=1, col=1 # Moved row/col outside go.Scatter
                )

        # Plot RSI subplot
        if 'RSI' in df.columns:
            fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], name="RSI", mode='lines', line=dict(color='darkblue')), row=2, col=1)
            fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1, annotation_text="RSI 30", annotation_position="bottom right")
            fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1, annotation_text="RSI 70", annotation_position="top right")
            fig.update_yaxes(title_text="RSI", range=[0, 100], row=2, col=1) # Set fixed RSI range

        # Plot Stochastic Oscillator subplot
        if 'Stoch_K' in df.columns and 'Stoch_D' in df.columns:
            fig.add_trace(go.Scatter(x=df.index, y=df['Stoch_K'], name="Stoch %K", mode='lines', line=dict(color='darkgreen')), row=3, col=1)
            fig.add_trace(go.Scatter(x=df.index, y=df['Stoch_D'], name="Stoch %D", mode='lines', line=dict(color='red', dash='dot')), row=3, col=1)
            fig.add_hline(y=20, line_dash="dash", line_color="green", row=3, col=1, annotation_text="Stoch 20", annotation_position="bottom right")
            fig.add_hline(y=80, line_dash="dash", line_color="red", row=3, col=1, annotation_text="Stoch 80", annotation_position="top right")
            fig.update_yaxes(title_text="Stochastic", range=[0, 100], row=3, col=1) # Set fixed Stochastic range

        fig.update_layout(
            title_text=f"<b>{symbol} - Trading Signals and Indicators</b>",
            height=900,
            showlegend=True,
            hovermode="x unified",
            xaxis_rangeslider_visible=False, # Hide rangeslider for cleaner plot
            template="plotly_white" # Use a clean template
        )
        fig.update_xaxes(title_text="Date", row=3, col=1)
        fig.update_yaxes(title_text="Price", row=1, col=1)

        fig.show(renderer="colab")
        logger.info(f"Plot generated for {symbol}.")
        return fig

    except Exception as e:
        logger.error(f"Error creating plot for {symbol}: {str(e)}")
        return None

def setup_google_sheets():
    """
    Authenticates with Google Sheets and ensures the main spreadsheet and worksheets exist.
    """
    try:
        scope = [
            "https://spreadsheets.google.com/feeds",
            "https://www.googleapis.com/auth/drive"
        ]

        # Check if the JSON file exists
        if not os.path.exists(config.GDRIVE_JSON):
            logger.error(f"Google Sheets service account JSON file not found at: {config.GDRIVE_JSON}")
            logger.error("Please ensure the service account JSON file is uploaded to the root of your Google Drive and the path in config.GDRIVE_JSON is correct.")
            return None

        creds = ServiceAccountCredentials.from_json_keyfile_name(
            config.GDRIVE_JSON, scope)
        gc = gspread.authorize(creds)

        try:
            sh = gc.open(config.SHEET_NAME)
            logger.info(f"Opened existing Google Sheet: '{config.SHEET_NAME}'")
        except gspread.exceptions.SpreadsheetNotFound:
            logger.info(f"Spreadsheet '{config.SHEET_NAME}' not found. Creating a new one...")
            sh = gc.create(config.SHEET_NAME)
            # Give some time for the sheet to be created and propagate
            time.sleep(2)
            # IMPORTANT: You might need to manually share the newly created sheet with your service account email
            # or add a line like:
            # sh.share('your-service-account-email@your-project-id.iam.gserviceaccount.com', perm_type='user', role='writer')
            # after creation, if permissions issues persist.
            logger.info(f"Successfully created new Google Sheet: '{config.SHEET_NAME}'")

        # Ensure worksheets exist
        try:
            trade_log_ws = sh.worksheet(config.TRADE_LOG_TAB)
            logger.info(f"Worksheet '{config.TRADE_LOG_TAB}' found.")
        except gspread.exceptions.WorksheetNotFound:
            trade_log_ws = sh.add_worksheet(title=config.TRADE_LOG_TAB, rows="1000", cols="20")
            logger.info(f"Worksheet '{config.TRADE_LOG_TAB}' created.")

        try:
            summary_ws = sh.worksheet(config.SUMMARY_TAB)
            logger.info(f"Worksheet '{config.SUMMARY_TAB}' found.")
        except gspread.exceptions.WorksheetNotFound:
            summary_ws = sh.add_worksheet(title=config.SUMMARY_TAB, rows="100", cols="10")
            logger.info(f"Worksheet '{config.SUMMARY_TAB}' created.")

        try:
            analytics_ws = sh.worksheet(config.ANALYTICS_TAB)
            logger.info(f"Worksheet '{config.ANALYTICS_TAB}' found.")
        except gspread.exceptions.WorksheetNotFound:
            analytics_ws = sh.add_worksheet(title=config.ANALYTICS_TAB, rows="100", cols="10")
            logger.info(f"Worksheet '{config.ANALYTICS_TAB}' created.")

        return sh
    except Exception as e:
        logger.error(f"Error setting up Google Sheets: {str(e)}")
        return None

def update_google_sheet(worksheet, df, clear_existing=True):
    """
    Updates a Google Sheet worksheet with DataFrame content.
    """
    if df is None or df.empty:
        logger.warning(f"Empty DataFrame provided for worksheet '{worksheet.title}'. Skipping update.")
        return False
    try:
        if clear_existing:
            worksheet.clear()
            logger.info(f"Cleared existing data in worksheet '{worksheet.title}'.")
        gd.set_with_dataframe(worksheet, df, row=1, col=1, include_index=True)
        logger.info(f"Successfully updated worksheet '{worksheet.title}' with {len(df)} rows.")
        return True
    except Exception as e:
        logger.error(f"Error updating Google Sheet worksheet '{worksheet.title}': {str(e)}")
        return False

def send_telegram_message(message):
    """
    Sends a message to a Telegram chat.
    """
    if not config.TELEGRAM_TOKEN or not config.TELEGRAM_CHAT_ID:
        logger.warning("Telegram token or chat ID not configured. Skipping Telegram message.")
        return

    url = f"https://api.telegram.org/bot{config.TELEGRAM_TOKEN}/sendMessage"
    payload = {
        "chat_id": config.TELEGRAM_CHAT_ID,
        "text": message,
        "parse_mode": "HTML" # Allows for bold, italics, etc.
    }
    try:
        response = requests.post(url, data=payload, timeout=5)
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        logger.info(f"Telegram message sent: {message}")
    except requests.exceptions.RequestException as e:
        logger.error(f"Error sending Telegram message: {e}")



def main_algo_run():
    """
    Main function to run the algo-trading system:
    1. Fetches data
    2. Calculates indicators
    3. Generates and executes trades (backtesting)
    4. Trains ML model
    5. Logs results to Google Sheets
    6. Sends Telegram notifications
    """
    logger.info("--- Starting Algo-Trading System Run ---")
    send_telegram_message("🤖 Algo-Trading System: Starting daily run...")

    try:
        # Setup Google Sheets
        spreadsheet = setup_google_sheets()
        if not spreadsheet:
            logger.critical("Failed to set up Google Sheets. Exiting.")
            send_telegram_message("🚨 Algo-Trading System: Failed to set up Google Sheets. Check logs!")
            return

        trade_log_ws = spreadsheet.worksheet(config.TRADE_LOG_TAB)
        summary_ws = spreadsheet.worksheet(config.SUMMARY_TAB)
        analytics_ws = spreadsheet.worksheet(config.ANALYTICS_TAB)

        all_trades = []
        ml_accuracies = {}

        strategy = TradingStrategy()

        # Prepare data for all symbols
        all_stock_data = prepare_dataset(config.STOCKS, config.INTERVAL, config.LOOKBACK_DAYS)

        for symbol, df in all_stock_data.items():
            if df is None or df.empty:
                logger.warning(f"Skipping {symbol} due to no data or processing errors.")
                send_telegram_message(f"⚠️ Algo-Trading System: Skipping {symbol} due to data issues.")
                continue

            try:
                logger.info(f"Processing trading strategy for {symbol}...")
                df_signals = strategy.generate_signals(df.copy())

                if df_signals is None:
                    logger.warning(f"Could not generate signals for {symbol}. Skipping trade execution and plot.")
                    continue

                # Execute trades for the symbol
                symbol_trades_df = strategy.execute_trades(symbol, df_signals)
                if not symbol_trades_df.empty:
                    all_trades.append(symbol_trades_df)
                    buy_count = symbol_trades_df[symbol_trades_df['Type'] == 'BUY'].shape[0]
                    sell_count = symbol_trades_df[symbol_trades_df['Type'] == 'SELL'].shape[0]
                    logger.info(f"{symbol}: Executed {buy_count} BUY and {sell_count} SELL trades.")
                    send_telegram_message(f"📈 {symbol}: Generated {buy_count} BUY and {sell_count} SELL signals.")
                else:
                    logger.info(f"No trades executed for {symbol}.")
                    send_telegram_message(f"ℹ️ {symbol}: No trading signals generated or trades executed.")

                # Train ML model for the symbol
                logger.info(f"Training ML model for {symbol}...")
                ml_model, accuracy, report = train_ml_model(df, symbol)
                if ml_model:
                    ml_accuracies[symbol] = accuracy
                    logger.info(f"ML model for {symbol} trained with accuracy: {accuracy:.2%}")
                    send_telegram_message(f"📊 {symbol} ML Model Accuracy: {accuracy:.2%}")
                else:
                    logger.warning(f"ML model training skipped for {symbol}.")
                    send_telegram_message(f"❌ {symbol} ML Model Training Skipped.")

                # Generate plot
                plot_trading_signals(symbol, df, symbol_trades_df)

            except Exception as e:
                logger.error(f"Error processing {symbol}: {str(e)}")
                send_telegram_message(f"⚠️ Error processing {symbol}: {str(e)}")
                continue

        # Consolidate all trades
        final_trades_df = pd.DataFrame()
        if all_trades:
            final_trades_df = pd.concat(all_trades, ignore_index=True)
            logger.info(f"Total consolidated trades: {len(final_trades_df)}")

        # Calculate overall performance metrics
        overall_metrics = calculate_performance_metrics(final_trades_df)

        # Update Google Sheets
        try:
            if not final_trades_df.empty:
                final_trades_df['Date'] = pd.to_datetime(final_trades_df['Date'])
                final_trades_df = final_trades_df.sort_values(by='Date').reset_index(drop=True)

                for col in config.TRADE_LOG_COLUMNS_ORDER:
                    if col not in final_trades_df.columns:
                        final_trades_df[col] = np.nan

                final_trades_df = final_trades_df[config.TRADE_LOG_COLUMNS_ORDER]
                update_google_sheet(trade_log_ws, final_trades_df, clear_existing=True)
            else:
                trade_log_ws.clear()
                trade_log_ws.update_cell(1,1, "No trades executed during this run.")

            summary_data = pd.DataFrame([overall_metrics]).T.reset_index()
            summary_data.columns = ['Metric', 'Value']
            update_google_sheet(summary_ws, summary_data, clear_existing=True)

            if ml_accuracies:
                analytics_data = pd.DataFrame.from_dict(ml_accuracies, orient='index', columns=['ML Accuracy'])
                analytics_data.index.name = 'Symbol'
                update_google_sheet(analytics_ws, analytics_data, clear_existing=True)

        except Exception as e:
            logger.error(f"Error updating Google Sheets: {str(e)}")
            send_telegram_message("⚠️ Error updating Google Sheets. Check logs!")

        final_message = (
            f"✅ Algo-Trading System: Run Completed!\n\n"
            f"💰 Total PnL: {overall_metrics['Total PnL']:.2f} ({overall_metrics['PnL %']:.2f}%)\n"
            f"📊 Win Rate: {overall_metrics['Win Rate']:.2%}\n"
            f"📉 Max Drawdown: {overall_metrics['Max Drawdown']:.2f}%\n\n"
            f"Check Google Sheet '{config.SHEET_NAME}' for details."
        )
        send_telegram_message(final_message)
        logger.info("--- Algo-Trading System Run Finished ---")

    except Exception as e:
        logger.critical(f"Critical error in main algo run: {str(e)}")
        send_telegram_message(f"🚨 Critical error in algo-trading system: {str(e)}")

if __name__ == "__main__":
    main_algo_run()

Mounted at /content/drive


In [24]:
!find /content -name "*.ipynb"


/content/drive/MyDrive/Colab Notebooks/Copy of data_preprocessing_tools (1).ipynb
/content/drive/MyDrive/Colab Notebooks/Copy of simple_linear_regression (1).ipynb
/content/drive/MyDrive/Colab Notebooks/Copy of simple_linear_regression.ipynb
/content/drive/MyDrive/Colab Notebooks/Copy of multiple_linear_regression.ipynb
/content/drive/MyDrive/Colab Notebooks/Copy of polynomial_regression (1).ipynb
/content/drive/MyDrive/Colab Notebooks/Copy of polynomial_regression.ipynb
/content/drive/MyDrive/Colab Notebooks/Copy of support_vector_regression (1).ipynb
/content/drive/MyDrive/Colab Notebooks/Copy of support_vector_regression.ipynb
/content/drive/MyDrive/Colab Notebooks/Untitled0.ipynb
/content/drive/MyDrive/Colab Notebooks/Copy of Untitled0.ipynb
/content/drive/MyDrive/Colab Notebooks/Copy of data_preprocessing_tools.ipynb
/content/drive/MyDrive/Colab Notebooks/imagedetec.ipynb
/content/drive/MyDrive/Colab Notebooks/laplacian.ipynb
/content/drive/MyDrive/Colab Notebooks/Untitled1.ipynb
